In [1]:

! cd ../data/; FILELIST="200505 200504 200501 200430 200429 200428 200427 200426 200425 200424 200423 200422" ; for fecha in `echo $FILELIST`  ; do  FILE=${fecha}_cam_covid19.pdf ; echo $FILE:::::  ;  [ ! -f ../data/${FILE} ] && wget https://www.comunidad.madrid/sites/default/files/doc/sanidad/$FILE ; ls -altr $FILE ; done 

! ls -altr ../data/*cam*
! pwd 

200505_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 556266 may  5 09:09 200505_cam_covid19.pdf
200504_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 557086 may  3 23:41 200504_cam_covid19.pdf
200501_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 556974 may  1 20:11 200501_cam_covid19.pdf
200430_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 554446 abr 30 08:36 200430_cam_covid19.pdf
200429_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 557572 abr 29 09:53 200429_cam_covid19.pdf
200428_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 555864 abr 28 09:10 200428_cam_covid19.pdf
200427_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 556578 abr 27 09:03 200427_cam_covid19.pdf
200426_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 556778 abr 26 08:37 200426_cam_covid19.pdf
200425_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 554376 abr 25 09:31 200425_cam_covid19.pdf
200424_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 554914 abr 25 19:34 200424_cam_covid19.pdf
200423_cam_covid19.pdf:::::
-rw-r--r-- 1 root root 556637 ab

In [2]:
from tabula import read_pdf
import os 
import pandas as pd
import glob 

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.141-1.b16.el7_3.x86_64/jre"

def get_daily_data(fecha):
    col2str = {'dtype': str}
    kwargs = {'output_format': 'dataframe',
              'pandas_options': col2str,
              'stream': False}
    df_pdf =  read_pdf('../data/'+fecha+'_cam_covid19.pdf',pages='1',multiple_tables = True,**kwargs)

    
    df = df_pdf[0]
    #print(fecha, df.index, df.columns)
    df = df[df['Unnamed: 0'].notna()]
    df = df[(df['Unnamed: 0']=='HOSPITALES') | (df['Unnamed: 0'] == 'DOMICILIOS') | (df['Unnamed: 0'] == 'CENTROS SOCIOSANITARIOS') | (df['Unnamed: 0'] == 'OTROS LUGARES') | (df['Unnamed: 0'] == 'FALLECIDOS TOTALES')]
    df = df[['Unnamed: 0','Unnamed: 2']]
    df['Unnamed: 2'] = df['Unnamed: 2'].astype(str).str.replace(r".", '')
    df = df.T
    df.columns = df.iloc[0]
    df = df.iloc[1:]

    df['Fecha'] =  pd.to_datetime(fecha, format='%y%m%d')
    #del df['Unnamed: 0']
    #df = df.reset_index(drop=True)
    df = df.rename_axis(None)

    df.set_index('Fecha', inplace=True, drop=True)
    df.index
    df.dropna()
    #df = df.T
    return df

def get_all_data( ): 
    BLACKLIST = ["200429","200422"]
    
    df = pd.DataFrame()
    list_df = []
 
    pdf_list= sorted(glob.glob('../data/*_cam_covid19.pdf'), 
                     key=os.path.getmtime,
                     reverse=True )
    
    for pdf_file in pdf_list:
        # extract fecha from username , eg : ../data/2200422_cam_covid19.pdf
        fecha = pdf_file.split('/')[2].split('_')[0]
        if fecha not in BLACKLIST:
            print("processing", fecha)
            df = get_daily_data(fecha) 
        list_df.append(df)

    df = pd.concat(list_df)
    df = df.astype(int)
    df = df.drop_duplicates()

    df = df.sort_values(by=['Fecha'], ascending=True)

    df['HOSPITALES hoy'] = df['HOSPITALES'] - df['HOSPITALES'].shift(1)
    df['CENTROS SOCIOSANITARIOS hoy'] = df['CENTROS SOCIOSANITARIOS'] - df['CENTROS SOCIOSANITARIOS'].shift(1)
    df['FALLECIDOS TOTALES hoy'] = df['FALLECIDOS TOTALES'] - df['FALLECIDOS TOTALES'].shift(1)

    df = df.sort_values(by=['Fecha'], ascending=False)

    return df 

In [3]:
total =  get_all_data()

total

processing 200505


CSVParseError: Error failed to create DataFrame with different column tables.
Try to set `multiple_tables=True`or set `names` option for `pandas_options`. 
, caused by ParserError('Error tokenizing data. C error: Expected 1 fields in line 3, saw 5\n',)

In [ ]:
import pandas as pd
import io
import matplotlib.dates as mdates
from matplotlib import pyplot as plt



chart_df=total[['HOSPITALES hoy','CENTROS SOCIOSANITARIOS hoy','FALLECIDOS TOTALES hoy']]
chart_df.plot(legend=True,figsize=(13.5,9))

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.show()



In [ ]:
from IPython.display import display, HTML
HTML("<h2>Muertes medias diarias desde que la comunidad de Madrid publica datos</h2>")


In [ ]:
# Calculamos los incrementos medios, desde que tenemos fechas
df = total

medias_datos = pd.DataFrame((df.head(1).max(axis=0) - df.tail(1).max(axis=0) ) / df.shape[0] ).T[['HOSPITALES','DOMICILIOS','CENTROS SOCIOSANITARIOS','OTROS LUGARES','FALLECIDOS TOTALES']]
medias_datos

In [ ]:
from IPython.display import display, HTML
HTML("<h2>Muertes medias diarias desde el principio de la crisis</h2>")


In [ ]:
from datetime import date
df = total

inicio_crisis = pd.Timestamp('2020-03-09 00:00:00')
dia_mas_reciente = df.index[0]
dias_transcurridos_inicio_crisis = dia_mas_reciente - inicio_crisis

medias_inicio_crisis = pd.DataFrame((df.head(1).max(axis=0)) / dias_transcurridos_inicio_crisis.days ).T[['HOSPITALES','DOMICILIOS','CENTROS SOCIOSANITARIOS','OTROS LUGARES','FALLECIDOS TOTALES']]
medias_inicio_crisis
                                    